In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Define the data loading function (adapted from your Streamlit app)
def load_data():
    """Loads all e-commerce datasets from the specified file path."""
    # Define file paths
    path = "E-commerce/"
    
    # Load datasets
    # The corrected line
    orders = pd.read_csv(
        path + "orders_dataset.csv", 
        parse_dates=[
            "order_purchase_timestamp", 
            "order_approved_at",          # <-- Add this
            "order_delivered_carrier_date", # <-- Add this
            "order_delivered_customer_date",
            "order_estimated_delivery_date"
        ]
    )
    order_items = pd.read_csv(path + "order_items_dataset.csv")
    products = pd.read_csv(path + "products_dataset.csv")
    cat_trans = pd.read_csv(path + "product_category_name_translation.csv")
    customers = pd.read_csv(path + "customers_dataset.csv")
    payments = pd.read_csv(path + "order_payments_dataset.csv")
    reviews = pd.read_csv(path + "order_reviews_dataset.csv", parse_dates=["review_creation_date", "review_answer_timestamp"])
    mql = pd.read_csv(path + "marketing_qualified_leads_dataset.csv", parse_dates=["first_contact_date"])
    deals = pd.read_csv(path + "closed_deals_dataset.csv", parse_dates=["won_date"])
    geolocation = pd.read_csv(path + "geolocation_dataset.csv")
    sellers = pd.read_csv(path + "sellers_dataset.csv")
    
    return orders, order_items, products, cat_trans, customers, payments, reviews, mql, deals, geolocation, sellers

# --- Load all data into variables ---
orders, order_items, products, cat_trans, customers, payments, reviews, mql, deals, geolocation, sellers = load_data()

# --- Perform initial merge to create the 'items' DataFrame ---
# This is the same merge helper from your app
items = order_items.merge(products[["product_id", "product_category_name"]], on="product_id", how="left")
items = items.merge(cat_trans, on="product_category_name", how="left")

# --- Verification Step ---
# You can now use all these DataFrames for exploration.
# For example, let's display the first few rows of the 'items' DataFrame to confirm it's loaded.
print("Data loaded and initial merge complete. The 'items' DataFrame is ready:")
items.head()

Data loaded and initial merge complete. The 'items' DataFrame is ready:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_category_name_english
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,cool_stuff
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,pet_shop
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,furniture_decor
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,perfumery
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,garden_tools


In [6]:
# Code to run in your notebook
print(orders['order_status'].unique())

['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']


In [7]:
# --- 1. Filter for relevant orders and calculate stage durations ---
# Ensure all timestamps needed are present
completed_orders = orders[
    orders['order_status'] == 'delivered'
].dropna(subset=[
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date'
])

# Calculate durations in days
# Using .dt.total_seconds() / (24 * 3600) is more precise for timedelta
completed_orders['processing_time'] = (completed_orders['order_approved_at'] - completed_orders['order_purchase_timestamp']).dt.total_seconds() / (24 * 3600)
completed_orders['seller_handling_time'] = (completed_orders['order_delivered_carrier_date'] - completed_orders['order_approved_at']).dt.total_seconds() / (24 * 3600)
completed_orders['carrier_shipping_time'] = (completed_orders['order_delivered_customer_date'] - completed_orders['order_delivered_carrier_date']).dt.total_seconds() / (24 * 3600)

# --- 2. Merge with product category information ---
# We use the 'items' helper DataFrame you already created
order_stage_times = completed_orders.merge(items[['order_id', 'product_category_name_english']], on='order_id', how='left')

# Drop orders that might have multiple different item categories to keep the analysis simple
order_stage_times = order_stage_times.drop_duplicates(subset=['order_id'])


# --- 3. Aggregate to find average times per category ---
# Focusing on the top 10 categories by revenue for clarity
top_10_cats = items.groupby('product_category_name_english')['price'].sum().nlargest(10).index

# Filter for top 10 categories
avg_times_by_cat = order_stage_times[
    order_stage_times['product_category_name_english'].isin(top_10_cats)
].groupby('product_category_name_english').agg({
    'processing_time': 'mean',
    'seller_handling_time': 'mean',
    'carrier_shipping_time': 'mean'
}).reset_index()


# --- 4. Prepare data for plotting and visualize ---
# 'Melt' the DataFrame to make it suitable for Plotly Express
avg_times_melted = avg_times_by_cat.melt(
    id_vars='product_category_name_english',
    value_vars=['processing_time', 'seller_handling_time', 'carrier_shipping_time'],
    var_name='delivery_stage',
    value_name='average_days'
)

# Create the stacked bar chart
import plotly.express as px

fig = px.bar(
    avg_times_melted,
    x='product_category_name_english',
    y='average_days',
    color='delivery_stage',
    title='Average Delivery Stage Duration by Product Category (Top 10)',
    labels={
        'product_category_name_english': 'Product Category',
        'average_days': 'Average Duration (Days)',
        'delivery_stage': 'Delivery Stage'
    },
    category_orders={ # Sort the stages logically
        'delivery_stage': ['processing_time', 'seller_handling_time', 'carrier_shipping_time']
    }
)

fig.show()

In [ ]:
# To get the total count of unique cities
city_count = geolocation['geolocation_state'].nunique()
print(f"There are {city_count} unique city names in the geolocation table.")

# To list the first 15 unique city names as an example
print("\nSample of city names:")
print(geolocation['geolocation_city'].unique()[:15])

There are 73 unique city names in the geolocation table.

Sample of city names:
['sao paulo' 'são paulo' 'sao bernardo do campo' 'jundiaí'
 'taboão da serra' 'sãopaulo' 'sp' 'sa£o paulo' 'sao jose dos campos'
 'osasco' 'carapicuíba' 'carapicuiba' 'barueri' 'santana de parnaiba'
 'pirapora do bom jesus']
